## AMATH Computational Data Analysis
### CP-4
#### Manjaree Binjolkar

In [1]:
import numpy as np
import scipy.fft
import scipy.io as sio
import copy
import scipy
from mpl_toolkits import mplot3d
#%matplotlib inline
import matplotlib.pyplot as plt
import pywt

In [2]:
def dc_wavelet(dcfile):
    # Input: dog/cat matrix
    # Output: edge detection
    
    m, n = dcfile.shape # 4096 x 80
    pxl = int(np.sqrt(m)) # cus images are square
    nw = m // 4 # wavelet resolution cus downsampling
    dcData = np.zeros((nw, n))
    
    for k in range(n):
        X = dcfile[:, k].reshape(pxl, pxl).astype('float')
        cA, (cH, cV, cD) = pywt.dwt2(X, 'haar') # only want horizontal and vertical
        cod_cH1 = np.abs(cH).astype('float') # horizontal rescaled
        cod_cV1 = np.abs(cV).astype('float') # vertical rescaled
        cod_edge = (cod_cH1 + cod_cV1) / 2 # edge detection
        cod_edge = cod_edge.reshape(nw, 1)
        dcData[:, k] = cod_edge.flatten()
    
    return dcData

In [3]:
#Load training data
training_images = sio.loadmat('CP4_training_images.mat')
training_labels = sio.loadmat('CP4_training_labels.mat')

training_images = training_images['training_images']
training_labels = training_labels['training_labels']

In [4]:
#Reshape training_images.m to 784x30000 in order to be consistent with the
#codes from the lecture.
training_images = training_images.reshape(784,30000, order ='f')

In [5]:
#Projecting onto principal components
#Conduct the wavelet transform on the entire training dataset.  To make it
#reusable you can write it as a function very similar (almost exactly the
#same) as dc_wavelet.m from the lecture.  For MATLAB you will need to
#include the function at the end of the code.

#Perform wavelet transform
#wavelet = dc_wavelet(training_images)
wavelet_temp = sio.loadmat('Training_DWT.mat')
wavelet = wavelet_temp['Training_DWT']
#print(wavelet.shape)
#wavelet[0:10, 0:10]

In [6]:
#Checking if the Training_DWT and dc_wavelet() are the same
#wavelet_dc = dc_wavelet(training_images)
#print(wavelet_dc.shape)
#wavelet_dc[0:10, 0:10]

In [7]:
#Find the SVD of the transformed data just like in Week7_LDA.m
u_t, s_t, v_t = np.linalg.svd(wavelet, full_matrices=False)

#Plot singular values (include in report, but not in gradescope submission)
#fig = plt.figure()
#plt.semilogy(s_t[0:20], ".")
#plt.xlabel("Index")
#plt.ylabel("Singular values")
#plt.show()

In [8]:
#type(u_t[0:15].T.shape)

In [9]:
#How many features (i.e., singular values) should we use?  Save this as
#A1.  Hint: it's less than what we used in the lecture

A1 = 15#; % 1x1. The number of PCA modes we are going to project onto.


#Project onto the principal components just like we did in Week7_LDA.m
#Restrict the matrix U to that of the feature space (like we did in 
#dc_trainer.m).  Save this as A2
A2 = u_t[:,0:15]#; % 196x15???
#A2.shape

In [10]:
#training_images[:,1].shape

In [11]:
#wavelet.shape

In [12]:
#Pick two numbers to train/test on.  Use 0 and 1 for the autograder.

#This is going to be quite different from what we did in the lectures.  In
#the lecture we had two datasets with dogs and cats.  Here everything is
#jumbled up so we need to separate them out.  Separate all the training 
#images of 0's and 1's using the training labels.  Hint: a for loop and
#some if statements should be sufficient.
training_0 = []
training_1 = []
y = A2.T@wavelet#projected data?????
#y = np.diag(s_t)@v_t
#print(y)

In [13]:
for i in range (len(training_labels)):
    if (training_labels[i] == 0):
        training_0.append(y[:,i])
        
    elif (training_labels[i] == 1):
        training_1.append(y[:,i])

In [14]:
training_0 = np.asarray(training_0).T
training_1 = np.asarray(training_1).T

In [15]:
nd = training_0.shape[1] #what will nd and nc be here?
nc = training_1.shape[1]
#dogs = 0 refer to the lectures
#cats = 1
md = np.mean(training_0, axis=1)
mc = np.mean(training_1, axis=1)

Sw = np.zeros((training_0.shape[0], training_0.shape[0]))
for k in range(nd):
    Sw += np.outer(training_0[:, k] - md, training_0[:, k] - md)
for k in range(nc):
    Sw += np.outer(training_1[:, k] - mc, training_1[:, k] - mc)

Sb = np.outer(md - mc, md - mc)

In [16]:
#Calculate the within class and between class variances just like in 
#Week7_LDA.m.  Save these as A3 and A4.

A3 = Sw#; % 15x15
A4 = Sb#; % 15x15
#print(A4.shape)

In [17]:
#Find the best projection line just like in Week7_LDA.m.  Save the
#normalized projection line w as A5
# Perform linear discriminant analysis

D, V2 = scipy.linalg.eig(Sb, Sw)
#print(D)

# Get the eigenvector corresponding to the largest eigenvalue
ind = np.argmax(np.abs(D))
w = V2[:,ind]

# Normalize the eigenvector
w = w / np.linalg.norm(w)
w = -w
A5_temp = w.reshape(15,1)#; % 15x1
A5 = A5_temp.copy()
#print(A5.shape)
#A5

In [18]:
#training_0

In [19]:
#plt.hist(vtraining_0)
#plt.hist(vtraining_1)

In [20]:
#Project the training data onto w just like in Week7_LDA.m
#w = -w
vtraining_0 = w.T @ training_0
vtraining_1 = w.T @ training_1

#Find the threshold value just like in Week7_LDA.m.  Save it as A6
# Sort the dog and cat projections in ascending order
sort0 = np.sort(vtraining_0)
sort1 = np.sort(vtraining_1)

# Initialize indices to find the threshold
t1 = len(sort0)-1   # start on the right
t2 = 0  # start on the left

# Move the indices towards each other until they pass
while sort0[t1] > sort1[t2]:
    t1 -= 1
    t2 += 1
    #print(t1,t2,sort0[t1],sort1[t2] )

# Calculate the threshold as the midpoint between the last dog value and the first cat value
threshold = (sort0[t1] + sort1[t2]) / 2
#print(sort0[t1])
#print(sort1[t2])
A6 = threshold.copy()#; % 1x1
#A6

In [21]:
#A6

In [22]:
#Classify test data

#Load test data
test_images = sio.loadmat('CP4_test_images.mat')
test_labels = sio.loadmat('CP4_test_labels.mat')

test_images = test_images['test_images']
test_labels = test_labels['test_labels']

In [23]:
#Reshape test_images.m to 784x5000 in order to be consistent with the
#codes from the lecture.
test_images = test_images.reshape(784,5000)

test_images_filtered = []
for i in range (5000):
    if (test_labels[i] == 0) or (test_labels[i] == 1):
        test_images_filtered.append(test_images[:,i])

In [24]:
test_images_filtered = np.asarray(test_images_filtered).T

In [25]:
test_images_filtered.shape

(784, 1062)

In [26]:
#From the test set pick out the 0's and 1's without revealing the labels.
#Save only the images of 0's and 1's as a new dataset and save the
#associated labels for those exact 0's and 1's as a new vector.
TestSet = test_images_filtered
TestNum = TestSet.shape[1]

#Wavelet transform:  you can just use the same function you did for the
#training portion.
Test_wave = dc_wavelet(TestSet) # dc_wavelet is a defined function

#Find the SVD of the transformed data just like in Week7_LDA.m
#U_test, S_test, V_test = np.linalg.svd(Test_wave, full_matrices=False)

#Project the test data onto the principal components just like in
#Week7_Learning.m
TestMat = A2.T@Test_wave # PCA projection
#TestMat = TestMat[:,0:15]
#print(TestMat.shape)
#w = w.reshape(15,1)
#print(w.shape)
pval = np.dot(w,TestMat) # Project onto w vector


In [27]:
#Checking performance just like we did in Week7_Learning.m.  If you did
#everything like I did (which may or may not be optimal), you should
#have a success rate of 0.9972.
# Cat = 1, dog = 0 for ResVec

ResVec_bool = pval > threshold
#print(ResVec_bool.shape)

ResVec = 1*ResVec_bool
#print(ResVec.shape)

In [28]:
#ResVec

In [29]:
#Save the results in a vector (just like in Week7_Learning.m) and save it
#as A7.

A7 = ResVec.reshape(1,1062) #; % 1x1062

#print(A7.shape)
#print(A7)

In [30]:
hiddenlabels = []

for i in range (len(test_labels)):
    if (test_labels[i] == 0) or (test_labels[i] == 1):
        hiddenlabels.append(test_labels[i])
        
hiddenlabels = np.asarray(hiddenlabels)    
#print(hiddenlabels.shape)

In [31]:
# 0s are correct and 1s are incorrect
err = np.abs(ResVec.reshape(1062,1) - hiddenlabels) #hiddenlabels=test_labels? what length to take test_labels
#do we need to slice test_label?
errNum = np.sum(err)
#print(errNum)
#print(TestNum)
sucRate = 1 - errNum/TestNum
sucRate

0.4708097928436912

In [32]:
#For report only, not for the autograder:  Now write an algorithm to
#classify all 10 digits.  One way to do this is by using the "one vs all
#" method; i.e., loop through the digits and conduct LDA on each digit
#vs. all the other digits.

In [33]:
#dog = sio.loadmat('dogData.mat')
#dog = dog['dog']
#dog

In [34]:

#fig = plt.figure()
#fig.suptitle('First Four Principal Components', fontsize=14, fontweight='bold')
#for k in range(1, 5):
#    ax = fig.add_subplot(2, 2, k)
#    ut1 = np.reshape(u_t[:, k-1], (14, 14))
#    ut2 = np.interp(ut1, (ut1.min(), ut1.max()), (0, 1))
#    ax.imshow(ut2, cmap='gray')
#plt.show()


In [35]:
#fig = plt.figure()
#fig.suptitle('Training Numbers Grid', fontsize=14, fontweight='bold')
#for k in range(1, 10):
#    ax = fig.add_subplot(3, 3, k)
#    training_images1 = np.reshape(training_images[:, k-1], (28, 28)).T
#    #needs transpose??
#    ax.imshow(training_images1)
#plt.show()


In [36]:
#fig = plt.figure()
#fig.suptitle('Test Numbers Grid', fontsize=14, fontweight='bold')
#for k in range(1, 10):
#    ax = fig.add_subplot(3, 3, k)
#    test_images1 = np.reshape(test_images[:, k-1], (28, 28))
#    ax.imshow(test_images1)
#plt.show()